In [1]:
# 기사랑 채권분석 라벨링 된 것 파일 불러오기
import pandas as pd
df_trainset = pd.read_csv('labeled_concat_interest_bond.csv')

In [2]:
df_trainset.columns

Index(['date', 'text', 'n_gram', 'callrate', '한달전date', '한달전callrate',
       'Labelling'],
      dtype='object')

In [3]:
df_trainset = df_trainset[['date', 'text', 'n_gram','Labelling']]
df_trainset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203948 entries, 0 to 203947
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   date       203948 non-null  object
 1   text       202661 non-null  object
 2   n_gram     202747 non-null  object
 3   Labelling  203948 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


In [4]:
df_trainset = df_trainset[df_trainset['n_gram'] != '[]'] #엔그램 없는 것 빼기

In [5]:
df_trainset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201528 entries, 0 to 203947
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   date       201528 non-null  object
 1   text       200327 non-null  object
 2   n_gram     200327 non-null  object
 3   Labelling  201528 non-null  object
dtypes: object(4)
memory usage: 7.7+ MB


In [6]:
201528 - 200327 # text가 없는 날짜 제거하기

1201

In [7]:
df_trainset = df_trainset[df_trainset['n_gram'].notnull()]
df_trainset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200327 entries, 0 to 203947
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   date       200327 non-null  object
 1   text       200327 non-null  object
 2   n_gram     200327 non-null  object
 3   Labelling  200327 non-null  object
dtypes: object(4)
memory usage: 7.6+ MB


In [8]:
from tqdm.notebook import tqdm

In [9]:
from collections import defaultdict
n_gram = defaultdict(lambda:[0,0,0]) # total_count, up_count, down_count

for i in tqdm(range(len(df_trainset))):
    texts = df_trainset['n_gram'].iloc[i].replace("[","").replace("]","").replace("'","").replace(",","")
    texts = texts.split()
    for text in texts:
        n_gram[text][0] += 1
        if df_trainset['Labelling'].iloc[i] == 'up':
            n_gram[text][1] += 1
        elif df_trainset['Labelling'].iloc[i] == 'down':
            n_gram[text][2] += 1

In [10]:
ngram, total_count, up_count, down_count = [],[],[],[]
for i, doc in n_gram.items():
    ngram.append(i)
    total_count.append(doc[0])
    up_count.append(doc[1])
    down_count.append(doc[2])

In [11]:
df_polar_score = pd.DataFrame([ngram, total_count, up_count, down_count],index=['ngram','total_count','up_count','down_count'])
df_polar_score = df_polar_score.T

In [12]:
# 총 개수가 15이하인 것 delete
df_polar_score = df_polar_score[df_polar_score['total_count']>=15]

In [13]:
# down_count가 0인 것 확인하기---------------------------------------------------------------------
# bayes인 것 확인 팀원들한테
df_polar_score.info()
df_polar_score[df_polar_score['down_count']==0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48896 entries, 0 to 145938
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ngram        48896 non-null  object
 1   total_count  48896 non-null  object
 2   up_count     48896 non-null  object
 3   down_count   48896 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


,ngram,total_count,up_count,down_count
8575,축연/NNG,123,123,0
10311,당지/NNG,19,18,0
19375,레트/NNG,21,18,0
24726,긴급조정권/NNG,23,23,0
25307,예상/NNG;채권시장/NNG;강세/NNG,17,17,0
...,...,...,...,...
132904,비즈니스/NNG;금리/NNG;인상/NNG,17,1,0
133303,깔대/VV,19,7,0
139548,감계/NNG,28,28,0
142809,경제/NNG;전망/NNG;단기/NNG;위험/NNG;감소/NNG,21,3,0


In [14]:
# 스팸 필터링 만들기
import numpy as np

df_polar_score['bayes_up'] = np.zeros(len(df_polar_score))
df_polar_score['bayes_down'] = np.zeros(len(df_polar_score))

total_up = df_polar_score['up_count'].sum()
total_down = df_polar_score['down_count'].sum()

for i in range(len(df_polar_score)):
    df_polar_score['bayes_up'].iloc[i] = (0.5 + df_polar_score['up_count'].iloc[i])/ (2 *0.5 + total_up)
    df_polar_score['bayes_down'].iloc[i] = (0.5 + df_polar_score['down_count'].iloc[i]) / (2 *0.5 + total_down)

C:\Users\180309\anaconda3\envs\mulcam_b\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
df_polar_score['polarity_score'] = df_polar_score['bayes_up']/df_polar_score['bayes_down']
df_polar_score['polarity'] = np.zeros(len(df_polar_score))

for i in range(len(df_polar_score)):
    if df_polar_score['polarity_score'].iloc[i] > 1.3:
        df_polar_score['polarity'].iloc[i] = 'Hawkish'
    elif df_polar_score['polarity_score'].iloc[i] < (1/1.3):
        df_polar_score['polarity'].iloc[i] = 'Dovish' 
    else:
        df_polar_score['polarity'].iloc[i] = 'Neutral'
df_polar_score[:10]

,ngram,total_count,up_count,down_count,bayes_up,bayes_down,polarity_score,polarity
0,은행/NNG,133309,55715,53252,0.003149,0.003214,0.979816,Neutral
1,수수료/NNG,13853,5458,5410,0.000309,0.000327,0.944810,Neutral
2,수입/NNG,16050,6846,6011,0.000387,0.000363,1.066582,Neutral
3,급급/NNG,320,127,128,0.000007,0.000008,0.929214,Neutral
4,일단/MAG,10132,4254,4005,0.000240,0.000242,0.994719,Neutral
5,팔/VV,13295,5051,5414,0.000286,0.000327,0.873717,Neutral
6,상품/NNG,47915,19554,19061,0.001105,0.001151,0.960723,Neutral
7,판매/NNG,37915,16369,14039,0.000925,0.000847,1.091924,Neutral
8,경쟁/NNG,13490,5854,5209,0.000331,0.000314,1.052452,Neutral
9,나서/VV,47340,18343,19154,0.001037,0.001156,0.896850,Neutral


In [16]:
df_hawkish = df_polar_score[df_polar_score['polarity']=='Hawkish']
df_dovish = df_polar_score[df_polar_score['polarity']=='Dovish']
len(df_hawkish), len(df_dovish)

(12521, 14127)

In [17]:
df_hawkish.to_csv('version1_df_hawkish_dict.csv',index=False)
df_dovish.to_csv('version1_df_dovish_dict.csv',index=False)